<a href="https://colab.research.google.com/github/hooriakhoramaki/Diabetes/blob/main/Copy_of_Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install scikit-fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 16.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# تعریف ورودی‌ها
fpg = ctrl.Antecedent(np.arange(0, 201, 1), 'fpg')  # قند خون ناشتا
hba1c = ctrl.Antecedent(np.arange(0, 11, 0.1), 'hba1c')  # HbA1c
ogtt = ctrl.Antecedent(np.arange(0, 301, 1), 'ogtt')  # تست OGTT
pregnant = ctrl.Antecedent(np.arange(0, 2, 1), 'pregnant')  # وضعیت بارداری (0 برای خیر، 1 برای بله)
exterim_thirst = ctrl.Antecedent(np.arange(0, 5, 1), 'exterim_thirst')  # وضعیت تشنگی شدید)
loss_wieght = ctrl.Antecedent(np.arange(0, 21, 1), 'loss_wieght')  # وضعیت از دست دادن وزن)

# تعریف خروجی
diabetes_type = ctrl.Consequent(np.arange(0, 5, 1), 'diabetes_type')

# مجموعه‌های فازی برای FPG
fpg['normal'] = fuzz.trimf(fpg.universe, [0, 50, 100])
fpg['prediabetes'] = fuzz.trimf(fpg.universe, [90, 110, 130])
fpg['diabetes'] = fuzz.trimf(fpg.universe, [120, 160, 200])

# مجموعه‌های فازی برای HbA1c
hba1c['normal'] = fuzz.trimf(hba1c.universe, [0, 3, 5.7])
hba1c['prediabetes'] = fuzz.trimf(hba1c.universe, [5, 6.0, 6.4])
hba1c['diabetes'] = fuzz.trimf(hba1c.universe, [6, 8.0, 10.0])

# مجموعه‌های فازی برای OGTT
ogtt['normal'] = fuzz.trimf(ogtt.universe, [0, 70, 140])
ogtt['prediabetes'] = fuzz.trimf(ogtt.universe, [130, 165, 200])
ogtt['diabetes'] = fuzz.trimf(ogtt.universe, [190, 250, 300])

# مجموعه‌های فازی برای وضعیت بارداری
pregnant['no'] = fuzz.trimf(pregnant.universe, [0, 0, 1])
pregnant['yes'] = fuzz.trimf(pregnant.universe, [0, 1, 1])

#مجموعه فازی برای کاهش وزن
loss_wieght['low'] = fuzz.trimf(loss_wieght.universe, [0, 2, 4])
loss_wieght['moderate'] = fuzz.trimf(loss_wieght.universe, [2, 5, 8])
loss_wieght['high'] = fuzz.trimf(loss_wieght.universe, [4, 10, 20])

#مجموعه فازی برای تشنگی شدید
exterim_thirst['low'] = fuzz.trimf(exterim_thirst.universe, [0, 1, 2])
exterim_thirst['moderate'] = fuzz.trimf(exterim_thirst.universe, [1, 2, 3])
exterim_thirst['high'] = fuzz.trimf(exterim_thirst.universe, [2, 3, 4])


# مجموعه‌های فازی برای خروجی (نوع دیابت)
diabetes_type['normal'] = fuzz.trimf(diabetes_type.universe, [0, 0, 1])
diabetes_type['prediabetes'] = fuzz.trimf(diabetes_type.universe, [0, 1, 2])
diabetes_type['type_1'] = fuzz.trimf(diabetes_type.universe, [1, 2, 3])
diabetes_type['type_2'] = fuzz.trimf(diabetes_type.universe, [2, 3, 3])
diabetes_type['gestational'] = fuzz.trimf(diabetes_type.universe, [3, 4, 4])

# تعریف قوانین فازی
rule1 = ctrl.Rule(fpg['normal'] & hba1c['normal'] & ogtt['normal'] & pregnant['no'], diabetes_type['normal'])
rule2 = ctrl.Rule(fpg['prediabetes'] | hba1c['prediabetes'] | ogtt['prediabetes'], diabetes_type['prediabetes'])
rule3 = ctrl.Rule(fpg['diabetes'] & hba1c['diabetes'] & ogtt['diabetes'] & pregnant['no'], diabetes_type['type_2'])
rule4 = ctrl.Rule(fpg['diabetes'] & hba1c['diabetes'] & pregnant['no'] & exterim_thirst['high'] &  loss_wieght['high'] , diabetes_type['type_1'])
rule5 = ctrl.Rule(fpg['diabetes'] | hba1c['diabetes'] | ogtt['diabetes'] & pregnant['yes'], diabetes_type['gestational'])
# ایجاد سیستم کنترل فازی
diabetes_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5])
diabetes_simulation = ctrl.ControlSystemSimulation(diabetes_ctrl)

# دریافت ورودی‌ها
fpg_input = float(input("قند خون ناشتا (FPG): "))
hba1c_input = float(input("HbA1c: "))
ogtt_input = float(input("OGTT: "))
pregnant_input = int(input("آیا باردار هستید؟ (1 برای بله, 0 برای خیر): "))
loss_weight_input = int(input("آیا دچار تشنگی میشوید ؟ (بین 0-4): "))
exterim_thirst_input = int(input("آیا کاهش وزن شدید دارید ؟ (بین 0- 20): "))

# تنظیم مقادیر ورودی
diabetes_simulation.input['fpg'] = fpg_input
diabetes_simulation.input['hba1c'] = hba1c_input
diabetes_simulation.input['ogtt'] = ogtt_input
diabetes_simulation.input['pregnant'] = pregnant_input
diabetes_simulation.input['loss_wieght'] = loss_weight_input
diabetes_simulation.input['exterim_thirst'] = exterim_thirst_input

# اجرای شبیه‌سازی
diabetes_simulation.compute()

# نمایش خروجی
output = diabetes_simulation.output['diabetes_type']
if output < 0.5:
    diagnosis = "Normal"
elif 0.5 <= output < 1.5:
    diagnosis = "Pre-Diabetes"
elif 1.5 <= output < 2.5:
    diagnosis = "Type 1 Diabetes"
elif 2.5 <= output < 3.5:
    diagnosis = "Type 2 Diabetes"
else:
    diagnosis = "Gestational Diabetes"

print("تشخیص: ", diagnosis)